In [17]:
from factfinder.calculate import Calculate
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

In [18]:
##INPUTS
pff_variable = 'mdnfinc'
geotype = 'CT20'
census_geoid_list = ['36005012102']

In [19]:
pd.options.display.float_format = "{:,.18f}".format

In [20]:
try:
    env_path = "../.env"
    load_dotenv(dotenv_path=env_path)
except:
    print(".env file is missing ...")

In [21]:
calculate = Calculate(
        api_key=os.environ["API_KEY"], year=2019, source="acs", geography='2010_to_2020'
    )

In [22]:
# See all digits of ratio
ratio = calculate.geo.ratio
print(ratio.dtypes)
ratio.loc[ratio.geoid_ct2020.isin(census_geoid_list), :]


geoid_ct2010     object
geoid_ct2020     object
ratio           float64
dtype: object


,geoid_ct2010,geoid_ct2020,ratio
4,36005001900,36005001901,0.245696400625978001
5,36005001900,36005001902,0.754303599374022027


In [23]:
# Get ranges and design factor from metadata
ranges = calculate.meta.median_ranges(pff_variable)
print(ranges)
design_factor = calculate.meta.median_design_factor(pff_variable)
print(f"\nDesign factor: {design_factor}")

{'mdhhiu10': [0, 9999], 'mdhhi10t14': [10000, 14999], 'mdhhi15t19': [15000, 19999], 'mdhhi20t24': [20000, 24999], 'mdhhi25t29': [25000, 29999], 'mdhhi30t34': [30000, 34999], 'mdhhi35t39': [35000, 39999], 'mdhhi40t44': [40000, 44999], 'mdhhi45t49': [45000, 49999], 'mdhhi50t59': [50000, 59999], 'mdhhi60t74': [60000, 74999], 'mdhhi75t99': [75000, 99999], 'mdhi100t124': [100000, 124999], 'mdhi125t149': [125000, 149999], 'mdhi150t199': [150000, 199999], 'mdhhi200pl': [200000, 9999999]}

Design factor: 1.5


In [24]:
# Calculate inputs in 2020 geogs
df = calculate.calculate_e_m_multiprocessing(list(ranges.keys()), geotype)
print(df.dtypes)
print(df.loc[df.census_geoid.isin(census_geoid_list), :])

census_geoid     object
pff_variable     object
geotype          object
e               float64
m               float64
dtype: object
  census_geoid pff_variable geotype                      e  \
4  36005001901     mdhhiu10    CT20  36.608763693270724104   
5  36005001902     mdhhiu10    CT20 112.391236306729283001   
4  36005001901   mdhhi10t14    CT20  21.375586854460088659   
5  36005001902   mdhhi10t14    CT20  65.624413145539918446   
4  36005001901   mdhhi15t19    CT20   7.616588419405318611   
5  36005001902   mdhhi15t19    CT20  23.383411580594682277   
4  36005001901   mdhhi20t24    CT20   7.370892018779341193   
5  36005001902   mdhhi20t24    CT20  22.629107981220659696   
4  36005001901   mdhhi25t29    CT20  13.513302034428791742   
5  36005001902   mdhhi25t29    CT20  41.486697965571210034   
4  36005001901   mdhhi30t34    CT20  13.021909233176835130   
5  36005001902   mdhhi30t34    CT20  39.978090766823164870   
4  36005001901   mdhhi35t39    CT20  19.655712050078243180  

In [25]:
# 3. create a pivot table with census_geoid as the index, and
# pff_variable as column names. df_pivoted.e -> the estimation dataframe
df_pivoted = df.loc[df.census_geoid.isin(census_geoid_list), ["census_geoid", "pff_variable", "e"]].pivot(
    index="census_geoid", columns="pff_variable", values=["e"]
)
print(df_pivoted.dtypes)
df_pivoted = df_pivoted.round(16)
print(df_pivoted)

   pff_variable
e  mdhhi10t14      float64
   mdhhi15t19      float64
   mdhhi200pl      float64
   mdhhi20t24      float64
   mdhhi25t29      float64
   mdhhi30t34      float64
   mdhhi35t39      float64
   mdhhi40t44      float64
   mdhhi45t49      float64
   mdhhi50t59      float64
   mdhhi60t74      float64
   mdhhi75t99      float64
   mdhhiu10        float64
   mdhi100t124     float64
   mdhi125t149     float64
   mdhi150t199     float64
dtype: object
                                 e                        \
pff_variable            mdhhi10t14            mdhhi15t19   
census_geoid                                               
36005001901  21.375586854460088659  7.616588419405318611   
36005001902  65.624413145539918446 23.383411580594682277   

                                                          \
pff_variable            mdhhi200pl            mdhhi20t24   
census_geoid                                               
36005001901   8.107981220657274335  7.370892018779341193 

In [26]:
# Empty dataframe to store the results
results = pd.DataFrame()
results["census_geoid"] = df_pivoted.index
results["pff_variable"] = pff_variable
results["geotype"] = geotype
results

,census_geoid,pff_variable,geotype
0,36005001901,mdhhinc,CT20
1,36005001902,mdhhinc,CT20


In [27]:
def get_median(ranges, row):
    ordered = list(ranges.keys())
    N = row[ordered].sum()
    print(f"\n\nN/2: {N/2}")
    C = 0
    i = 0
    while C <= N / 2 and i <= len(ranges.keys()) - 1:
        print(f"\nRange i: {i}")
        C += row[ordered[i]]
        print(f"Cumulative frequency C: {C}")
        i += 1
    i = i - 1
    if i == 0:
        print("N/2 is in first range")
        median = list(ranges.values())[0][1]
        print(f"Median: {median}")
    elif C == 0.0:
        print("Cumulative frequency is 0")
        median = 0.0
        print(f"Median: {median}")
    elif i == len(ranges.keys()) - 1:
        print("N/2 is in top range")
        median = list(ranges.values())[-1][0]
        print(f"Median: {median}")
    else:
        print(f"\nN/2 is in range {i}")
        print(f"Range {i}:", ranges[ordered[i]])
        C = C - row[ordered[i]]
        print(f"C_i-1: {C}")
        L = ranges[ordered[i]][0]
        print(f"L_i: {L}")
        F = row[ordered[i]]
        print(f"F_i: {F}")
        W = ranges[ordered[i]][1] - ranges[ordered[i]][0]
        print(f"W_i: {W}")
        median = L + (N / 2 - C) * W / F
        print(f"Median: {median}")
    return median

In [28]:
def get_median_moe(ranges, row, DF=1.1):
    md = row["e"]
    print("\n\n=======")
    print(f"Median: {md}\n")
    if md >= list(ranges.values())[-1][0]:
        print("Median is above top bin lower value")
        return np.nan
    else:
        ordered = list(ranges.keys())
        B = row[ordered].sum()
        if B == 0:
            print("Size of base is zero")
            return np.nan
        else:
            cumm_dist = list(np.cumsum(row[ordered]) / B * 100)
            print(f"Cumulative dist:\n {cumm_dist}")

            se_50 = DF * (((93 / (7 * B)) * 2500)) ** 0.5
            print(f"SE of 50%: {se_50}\n\n")

            if se_50 >= 50:
                return np.nan
            else:
                p_lower = 50 - se_50
                print(f"p_lower: {p_lower}")
                p_upper = 50 + se_50
                print(f"p_upper: {p_upper}")

                lower_bin = min([cumm_dist.index(i) for i in cumm_dist if i > p_lower])
                print(f"Bin containing p_lower: {lower_bin}")
                upper_bin = min([cumm_dist.index(i) for i in cumm_dist if i > p_upper])
                print(f"Bin containing p_upper: {upper_bin}")

                if lower_bin >= len(ordered) - 1:
                    return np.nan
                else:
                    if lower_bin == upper_bin:
                        print("\nBoth bounds are in the same bin\n")
                        A1 = min(ranges[ordered[lower_bin]])
                        print(f"Smallest value in the bin: {A1}")
                        A2 = min(ranges[ordered[lower_bin + 1]])
                        print(f"Largest value in the bin: {A2}")
                        C1 = cumm_dist[lower_bin - 1]
                        print(f"Cumulative percent of units less than smallest value: {C1}")
                        C2 = cumm_dist[lower_bin]
                        print(f"Cumulative percent of units less than largest value: {C2}")
                        lowerbound = (p_lower - C1) * (A2 - A1) / (C2 - C1) + A1
                        upperbound = (p_upper - C1) * (A2 - A1) / (C2 - C1) + A1
                        print(f"Confidence interval: {lowerbound} to {upperbound}")

                    else:
                        print("\nBounds are in different bins\n")
                        A1_l = min(ranges[ordered[lower_bin]])
                        print(f"Smallest value in the lower bin: {A1_l}")
                        A2_l = min(ranges[ordered[lower_bin + 1]])
                        print(f"Largest value in the lower bin: {A2_l}")
                        C1_l = cumm_dist[lower_bin - 1]
                        print(f"Cumulative percent of units less than lower bin smallest value: {C1_l}")
                        C2_l = cumm_dist[lower_bin]
                        print(f"Cumulative percent of units less than lower bin largest value: {C2_l}")

                        if upper_bin + 1 > len(ordered) - 1:
                            print("\nUpper bound is in top bin")
                            A1_u = min(ranges[ordered[upper_bin]])
                            print(f"Smallest value in the upper bin: {A1_u}")
                            A2_u = A1_u
                            print(f"Largest value in the upper bin: {A2_u}")
                            C1_u = cumm_dist[upper_bin - 1]
                            print(f"Cumulative percent of units less than upper bin smallest value: {C1_u}")
                            C2_u = cumm_dist[upper_bin]
                            print(f"Cumulative percent of units less than upper bin largest value: {C2_u}")
                        else:
                            print("\nUpper bound is below top bin")
                            A1_u = min(ranges[ordered[upper_bin]])
                            print(f"Smallest value in the upper bin: {A1_u}")
                            A2_u = min(ranges[ordered[upper_bin + 1]])
                            print(f"Largest value in the upper bin: {A2_u}")
                            C1_u = cumm_dist[upper_bin - 1]
                            print(f"Cumulative percent of units less than upper bin smallest value: {C1_u}")
                            C2_u = cumm_dist[upper_bin]
                            print(f"Cumulative percent of units less than upper bin largest value: {C2_u}")

                        lowerbound = (p_lower - C1_l) * (A2_l - A1_l) / (
                            C2_l - C1_l
                        ) + A1_l
                        upperbound = (p_upper - C1_u) * (A2_u - A1_u) / (
                            C2_u - C1_u
                        ) + A1_u
                        print(f"Confidence interval: {lowerbound} to {upperbound}")

                    print(f"MOE: {(upperbound - lowerbound) * 1.645 / 2}")
                    return (upperbound - lowerbound) * 1.645 / 2


In [29]:
# 4. calculate median estimation using get_median
results["e"] = (
    df_pivoted.e.loc[
        df_pivoted.e.index == results.census_geoid, list(ranges.keys())
    ]
    .apply(lambda row: get_median(ranges, row), axis=1)
    .to_list()
)



N/2: 132.55320813771516

Range i: 0
Cumulative frequency C: 36.608763693270724

Range i: 1
Cumulative frequency C: 57.98435054773081

Range i: 2
Cumulative frequency C: 65.60093896713613

Range i: 3
Cumulative frequency C: 72.97183098591547

Range i: 4
Cumulative frequency C: 86.48513302034426

Range i: 5
Cumulative frequency C: 99.50704225352109

Range i: 6
Cumulative frequency C: 119.16275430359934

Range i: 7
Cumulative frequency C: 132.18466353677618

Range i: 8
Cumulative frequency C: 145.20657276995303

N/2 is in range 8
Range 8: [45000, 49999]
C_i-1: 132.18466353677618
L_i: 45000
F_i: 13.021909233176835
W_i: 4999
Median: 45141.481132075474


N/2: 406.9467918622849

Range i: 0
Cumulative frequency C: 112.39123630672928

Range i: 1
Cumulative frequency C: 178.0156494522692

Range i: 2
Cumulative frequency C: 201.39906103286387

Range i: 3
Cumulative frequency C: 224.02816901408454

Range i: 4
Cumulative frequency C: 265.5148669796557

Range i: 5
Cumulative frequency C: 305.49295

In [30]:
# 5. Calculate median moe using get_median_moe
# Note that median moe calculation needs the median estimation
# so we seperated df_pivoted.m out as a seperate dataframe
e = df_pivoted.e.copy()
e["e"] = results.loc[e.index == results.census_geoid, "e"].to_list()
results["m"] = (
    e.loc[e.index == results.census_geoid, list(ranges.keys()) + ["e"]]
    .apply(lambda row: get_median_moe(ranges, row, design_factor), axis=1)
    .to_list()
)



Median: 45141.481132075474

Cumulative dist:
 [13.809082483781276, 21.872103799814642, 24.745134383688598, 27.525486561631133, 32.62279888785912, 37.53475440222427, 44.94902687673771, 49.86098239110287, 54.77293790546802, 61.260426320667285, 70.71362372567191, 84.52270620945319, 89.7126969416126, 95.8294717330862, 96.9416126042632, 100.0]
SE of 50%: 16.789725592885752


p_lower: 33.21027440711425
p_upper: 66.78972559288576
Bin containing p_lower: 5
Bin containing p_upper: 10

Bounds are in different bins

Smallest value in the lower bin: 30000
Largest value in the lower bin: 35000
Cumulative percent of units less than lower bin smallest value: 32.62279888785912
Cumulative percent of units less than lower bin largest value: 37.53475440222427

Upper bound is below top bin
Smallest value in the upper bin: 60000
Largest value in the upper bin: 75000
Cumulative percent of units less than upper bin smallest value: 61.260426320667285
Cumulative percent of units less than upper bin largest v

In [31]:
print(results.dtypes)
results.head()

census_geoid     object
pff_variable     object
geotype          object
e               float64
m               float64
dtype: object


,census_geoid,pff_variable,geotype,e,m
0,36005001901,mdhhinc,CT20,"45,141.481132075474306475","31,399.506005905746860662"
1,36005001902,mdhhinc,CT20,"45,141.481132075474306475","16,835.841853604146308498"


In [32]:
# Peform full calculation (including cleaning/rounding) to show display output

full_calc = calculate(pff_variable, geotype)
print(full_calc.dtypes)
print(full_calc.loc[full_calc.census_geoid.isin(census_geoid_list),:])

census_geoid     object
labs_geoid       object
geotype          object
labs_geotype     object
pff_variable     object
c               float64
e               float64
m               float64
p               float64
z               float64
dtype: object
  census_geoid labs_geoid geotype labs_geotype pff_variable  \
4  36005001901    2001901    CT20       CT2020      mdhhinc   
5  36005001902    2001902    CT20       CT2020      mdhhinc   

                      c                         e                         m  \
4 41.200000000000002842 46,366.000000000000000000 31,400.000000000000000000   
5 22.399999999999998579 45,762.000000000000000000 16,836.000000000000000000   

    p   z  
4 nan nan  
5 nan nan  
